In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from classifiers import LogisticRegression as LogReg
from classifiers import PyLogisticRegression as PyLogReg

In [3]:
logit = LogReg()

In [4]:
X = np.asarray([[i for i in range(1024)] for _ in range(1024)]).astype('float64')
y = np.random.choice([0, 1], 1024).astype('float64')

In [6]:
logit = LogReg(n_iter=1)

In [7]:
logit.fit(X,y)

AttributeError: 'LogisticRegression' object has no attribute 'theta'